# Tugas 03 : **Melakukan Pengujian dan Deploy**

NAMA : Mohammad Hasan Basri

NIM  : 210411100169

MATA KULIAH : Pencarian dan Penambangan Web - A

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pickle
import pandas as pd

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/My Drive/PPWA/report/Tugas-PPWA/Hasil_preprocesing.csv")
df.head()

,judul,tanggal,isi,kategori,cleansing,case_folding,tokenize,stopword_removal
0,"Nunggak 8 Bulan, Segini Pajak Ford Mustang Mil...","Kamis, 19 Sep 2024 13:05 WIB",Jakarta - Bareskrim Polri menyita aset senilai...,Otomotif,Jakarta Bareskrim Polri menyita aset senilai ...,jakarta bareskrim polri menyita aset senilai ...,"['jakarta', 'bareskrim', 'polri', 'menyita', '...",jakarta bareskrim polri menyita aset senilai r...
1,Bos Ford Kaget usai Jajal Mobil China: Mereka ...,"Kamis, 19 Sep 2024 12:33 WIB","Jakarta - Chief Executive Officer (CEO) Ford, ...",Otomotif,Jakarta Chief Executive Officer CEO Ford Jim ...,jakarta chief executive officer ceo ford jim ...,"['jakarta', 'chief', 'executive', 'officer', '...",jakarta chief executive officer ceo ford jim f...
2,"Tarif Tol Dalam Kota Naik Jadi Segini, Berlaku...","Kamis, 19 Sep 2024 12:08 WIB",Jakarta - Jasa Marga mengumumkan kenaikan tari...,Otomotif,Jakarta Jasa Marga mengumumkan kenaikan tarif...,jakarta jasa marga mengumumkan kenaikan tarif...,"['jakarta', 'jasa', 'marga', 'mengumumkan', 'k...",jakarta jasa marga mengumumkan kenaikan tarif ...
3,Pak RT Aleix Espargaro Tak Sabar Balapan Terak...,"Kamis, 19 Sep 2024 11:40 WIB","Jakarta - Pebalap Aprilia asal Spanyol, Aleix ...",Otomotif,Jakarta Pebalap Aprilia asal Spanyol Aleix Es...,jakarta pebalap aprilia asal spanyol aleix es...,"['jakarta', 'pebalap', 'aprilia', 'asal', 'spa...",jakarta pebalap aprilia spanyol aleix espargar...
4,Angkot Listrik Bakal Diuji Coba di Jakarta,"Kamis, 19 Sep 2024 11:18 WIB",Jakarta - PT Transportasi Jakarta (TransJakart...,Otomotif,Jakarta PT Transportasi Jakarta TransJakarta ...,jakarta pt transportasi jakarta transjakarta ...,"['jakarta', 'pt', 'transportasi', 'jakarta', '...",jakarta pt transportasi jakarta transjakarta k...


**TF-IDF (Term Frequency-Inverse Document Frequency)**


---


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['stopword_removal'])

# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)
# Menyimpan hasil ke file CSV
tfidf_df.to_csv('hasil_tfidf.csv', index=False)

In [ ]:
# Menggunakan kolom 'cleaned_text' sebagai teks input dan 'kategori' sebagai label
texts = df['stopword_removal'].fillna('')  # Mengganti NaN dengan string kosong jika ada
labels = df['kategori'].fillna('unknown')  # Mengganti NaN di kategori jika ada

# Melihat label yang digunakan untuk klasifikasi
print(labels.unique())

['Otomotif' 'Keuangan']


**mengubah data teks menjadi representasi numerik menggunakan TF-IDF Vectorizer, kemudian membagi dataset yang sudah diubah ini menjadi data latih dan data uji menggunakan fungsi train_test_split.**

In [ ]:
from sklearn.model_selection import train_test_split
# Melakukan transformasi TF-IDF
tfidf = TfidfVectorizer(max_features=None)  # Menggunakan 5000 fitur teratas
X_tfidf = tfidf.fit_transform(texts)

# Memisahkan dataset menjadi training dan testing
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, labels, test_size=0.2, random_state=42)

# Melihat bentuk dari hasil TF-IDF
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (80, 5471)
Shape of X_test: (20, 5471)


**membuat dan melatih model Logistic Regression dengan regularisasi untuk mencegah overfitting, kemudian menggunakan model tersebut untuk memprediksi hasil pada data uji.**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Membuat model Logistic Regression dengan regularisasi
model = LogisticRegression(max_iter=1000, C=0.3)  # Mengurangi nilai C untuk menambah regularisasi

# Melatih model dengan data yang sudah diresample (jika diperlukan)
model.fit(X_train, y_train)

# Memprediksi hasil pada data testing
y_pred = model.predict(X_test)

**Evaluasi Hasil**

---


In [ ]:
# Evaluasi hasil prediksi
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Menampilkan laporan klasifikasi
print(classification_report(y_test, y_pred))

Accuracy: 85.00%
              precision    recall  f1-score   support

    Keuangan       0.73      1.00      0.84         8
    Otomotif       1.00      0.75      0.86        12

    accuracy                           0.85        20
   macro avg       0.86      0.88      0.85        20
weighted avg       0.89      0.85      0.85        20



**Menyimpan model dan vectorizer (TF-IDF dan Logistic Regression) ke dalam file menggunakan pickle**

---


In [ ]:
# Menyimpan tfidf_matrix dan vectorizer menggunakan pickle
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)
# Menyimpan model ke dalam file pickle
with open('logistic_regression_model.pkl', 'wb') as file:
    pickle.dump(model, file)

**PENGUJIAN**

---



In [ ]:
import joblib

# Memuat vectorizer dan model
tfidf = joblib.load('tfidf_vectorizer.pkl')
model = joblib.load('logistic_regression_model.pkl')

In [ ]:
# Teks baru yang ingin diprediksi
new_text = ['''Jakarta, 25 September 2024 – Pertumbuhan sektor keuangan di Indonesia terus menunjukkan tren positif, terutama di tengah pemulihan ekonomi pasca-pandemi. Data dari Bank Indonesia (BI) menunjukkan peningkatan tajam dalam penggunaan layanan keuangan digital sepanjang tahun 2024, dengan transaksi e-wallet naik 35% dibandingkan periode yang sama tahun lalu.''']

# Preprocessing dan transformasi menggunakan TF-IDF yang sudah disimpan
new_text_tfidf = tfidf.transform(new_text)

# Melakukan prediksi menggunakan model Logistic Regression
prediction = model.predict(new_text_tfidf)

# Hasil prediksi
predicted_category = "pariwisata	" if prediction[0] == 'pariwisata	' else "Keuangan"
print(f"Prediksi: {predicted_category}")

Prediksi: Keuangan


In [ ]:
# Mengecek versi scikit-learn
import sklearn
print(sklearn.__version__)

1.5.2
